In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/thesahilmandal/Data-Science-Project-01.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "thesahilmandal"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "1715636383e3565534aabd6a753d213c363ed85a"

In [2]:
%pwd

'/workspaces/Data-Science-Project-01/research'

In [3]:
os.chdir("../")
%pwd

'/workspaces/Data-Science-Project-01'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [11]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/thesahilmandal/Data-Science-Project-01.mlflow"
        )
        return model_evaluation_config

In [13]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [20]:
import tempfile
import pandas as pd
import numpy as np
import joblib
import mlflow
from pathlib import Path
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Save model locally and log as artifact (no registry)
            with tempfile.TemporaryDirectory() as tmp_dir:
                model_path = Path(tmp_dir) / "model.pkl"
                joblib.dump(model, model_path)
                mlflow.log_artifact(str(model_path), artifact_path="model")


In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-13 07:08:08,220: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-13 07:08:08,224: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 07:08:08,227: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-13 07:08:08,227: INFO: common: created directory at: artifacts]
[2025-08-13 07:08:08,228: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-13 07:08:08,759: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
🏃 View run merciful-bat-489 at: https://dagshub.com/thesahilmandal/Data-Science-Project-01.mlflow/#/experiments/0/runs/549f9b4547e648c8968389c3d70174c0
🧪 View experiment at: https://dagshub.com/thesahilmandal/Data-Science-Project-01.mlflow/#/experiments/0
